In [9]:
from pathlib import Path

import numpy as np
import pandas
import rasterio
import scipy

In [2]:
def read_rp_map(fname: Path) -> np.ndarray:
    """Read flood map tiff to ndarray
    """
    with rasterio.open(fname) as dataset:
        data = dataset.read(1)
        data[data == dataset.nodata] = 0

    return data

def read_metadata(fname):
    with rasterio.open(fname) as dataset:
        crs = dataset.crs
        width = dataset.width
        height = dataset.height
        transform = dataset.transform
    return crs, width, height, transform

def save_to_tif(data, fname, crs, transform):
    with rasterio.open(
        fname,
        "w",
        driver="GTiff",
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=crs,
        transform=transform,
        compress="lzw",
    ) as dataset:
        dataset.write(data, 1)

def read_rp_maps(rp_maps: dict[float, Path]) -> dict[float, np.ndarray]:
    """Read flood map tiffs to dict of ndarrays
    """
    rp_data: dict[float, np.ndarray] = {}
    for rp, fname in rp_maps.items():
        rp_data[rp] = read_rp_map(fname)
    return rp_data

def interpolate_depth(rp: float, rp_l: float, depth_l: np.ndarray, rp_u: float, depth_u: np.ndarray) -> np.ndarray:
    """Interpolate between two flood map ndarrays
    """
    rp_factor = (np.log(rp) - np.log(rp_l)) / (np.log(rp_u) - np.log(rp_l))
    depth = depth_l + ((depth_u - depth_l) * rp_factor)

    return depth

def pick_upper_lower_rps(rp: float, rps: list[float]) -> tuple[float]:
    bin_index = np.searchsorted(rps, rp, side="left")
    rp_l = rps[bin_index - 1]
    rp_u = rps[bin_index]
    return rp_l, rp_u

def calculate_rp_maps(rps_to_calculate, rps_input):
    # Read all data
    depths = read_rp_maps(rps_input)
    baseline_rps = list(depths.keys())
    # Read metadata
    crs, width, height, transform = read_metadata(rps_input[baseline_rps[0]])
    depths[1e-3] = np.zeros((height, width))
    depths[2.0] = np.zeros((height, width))
    depths[1e6] = depths[max(baseline_rps)]
    baseline_rps = [1e-3, 2] + sorted(baseline_rps) + [1e6]

    for rp, output_fname in rps_to_calculate.items():
        rp_l, rp_u = pick_upper_lower_rps(rp, baseline_rps)
        print(rp_l, rp_u)
        depth = interpolate_depth(rp, rp_l, depths[rp_l], rp_u, depths[rp_u])
        save_to_tif(depth, output_fname, crs, transform)

In [3]:
rps_to_calculate = {
    2: "JM_FLRF_UD_Q2_RD_02-aligned.tif",
    5: "JM_FLRF_UD_Q5_RD_02-aligned.tif",
    10: "JM_FLRF_UD_Q10_RD_02-aligned.tif"
}
rps_input = {
    20: "JM_FLRF_UD_Q20_RD_02-aligned.tif",
    50: "JM_FLRF_UD_Q50_RD_02-aligned.tif",
    100: "JM_FLRF_UD_Q100_RD_02-aligned.tif",
    200: "JM_FLRF_UD_Q200_RD_02-aligned.tif",
    500: "JM_FLRF_UD_Q500_RD_02-aligned.tif",
}
calculate_rp_maps(rps_to_calculate, rps_input)

0.001 2
2 20
2 20


In [56]:
flow_reductions = pandas.DataFrame({
    'reduction_percent': [5, 10, 20, 30],
    'rp2': [2.4, 2.9, 4.8, 7.6],
    'rp10': [13,18,33,55.5],
    'rp50': [67,90,173,297],
    'rp100': [133,180,346,594],
    'rp200': [266,359,690,1188],
    'rp1000': [1327,1788,3419,5941],
})
# proportion of baseline flow
flow_reductions['flow'] = (1 - flow_reductions.reduction_percent / 100)

# interpolate RP20 values
rps = [2, 10, 50, 100, 200, 1000]
flows = flow_reductions['flow'].values

interpolator = scipy.interpolate.RegularGridInterpolator(
    (rps, flows),
    flow_reductions[['rp2', 'rp10', 'rp50', 'rp100', 'rp200', 'rp1000']].values.T,
    method='cubic'
)

rps_new = [[20]]
flow_reductions['rp20'] = interpolator((rps_new, [flows]))[0]

flow_reductions[['flow', 'rp2', 'rp10', 'rp20', 'rp50', 'rp100', 'rp200', 'rp1000']]

,flow,rp2,rp10,rp20,rp50,rp100,rp200,rp1000
0,0.95,2.4,13.0,26.422925,67,133,266,1327
1,0.90,2.9,18.0,36.386359,90,180,359,1788
2,0.80,4.8,33.0,68.162588,173,346,690,3419
3,0.70,7.6,55.5,115.760827,297,594,1188,5941


In [ ]:
for _, row in flow_reductions.iterrows():
    flow = row.flow
    # declare desired output RP and filename
    rps_to_calculate = {
        10: f"JM_FLRF_UD_Q10_RD_02_flow{flow}.tif",
        20: f"JM_FLRF_UD_Q20_RD_02_flow{flow}.tif",
        50: f"JM_FLRF_UD_Q50_RD_02_flow{flow}.tif",
        100: f"JM_FLRF_UD_Q100_RD_02_flow{flow}.tif",
    }
    # use baseline RP maps "mislabelled" as after change in flow
    rps_input = {
        row.rp2: "JM_FLRF_UD_Q2_RD_02-aligned.tif",
        row.rp10: "JM_FLRF_UD_Q10_RD_02-aligned.tif",
        row.rp20: "JM_FLRF_UD_Q20_RD_02-aligned.tif",
        row.rp50: "JM_FLRF_UD_Q50_RD_02-aligned.tif",
        row.rp100: "JM_FLRF_UD_Q100_RD_02-aligned.tif",
    }
    calculate_rp_maps(rps_to_calculate, rps_input)

{10: 'JM_FLRF_UD_Q10_RD_02_flow0.95.tif', 20: 'JM_FLRF_UD_Q20_RD_02_flow0.95.tif', 50: 'JM_FLRF_UD_Q50_RD_02_flow0.95.tif', 100: 'JM_FLRF_UD_Q100_RD_02_flow0.95.tif'}
{np.float64(2.4): 'JM_FLRF_UD_Q2_RD_02-aligned.tif', np.float64(13.0): 'JM_FLRF_UD_Q10_RD_02-aligned.tif', np.float64(26.422925397137593): 'JM_FLRF_UD_Q20_RD_02-aligned.tif', np.float64(67.0): 'JM_FLRF_UD_Q50_RD_02-aligned.tif', np.float64(133.0): 'JM_FLRF_UD_Q100_RD_02-aligned.tif'}
2.4 13.0
13.0 26.422925397137593
26.422925397137593 67.0
67.0 133.0
{10: 'JM_FLRF_UD_Q10_RD_02_flow0.9.tif', 20: 'JM_FLRF_UD_Q20_RD_02_flow0.9.tif', 50: 'JM_FLRF_UD_Q50_RD_02_flow0.9.tif', 100: 'JM_FLRF_UD_Q100_RD_02_flow0.9.tif'}
{np.float64(2.9): 'JM_FLRF_UD_Q2_RD_02-aligned.tif', np.float64(18.0): 'JM_FLRF_UD_Q10_RD_02-aligned.tif', np.float64(36.38635922911745): 'JM_FLRF_UD_Q20_RD_02-aligned.tif', np.float64(90.0): 'JM_FLRF_UD_Q50_RD_02-aligned.tif', np.float64(180.0): 'JM_FLRF_UD_Q100_RD_02-aligned.tif'}
2.9 18.0
18.0 36.38635922911745
3